In [ ]:
!pip install -Iv keras=='2.3.1'
!pip install -Iv tensorflow=='2.1.0'


In [ ]:

import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold

from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR

from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor

from sklearn.model_selection import cross_validate

import copy

import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 300
import seaborn as sns
from matplotlib.patches import PathPatch

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

import statistics
import os
from scipy.stats import shapiro
from scipy.stats import norm
import statistics
import keras

In [ ]:
# Import the data
dataset = pd.read_excel('data.xlsx',sheet_name='data')

In [ ]:
# Remove the outlier sample with the biggest porosity

placesConsidered = ["Pedreira_Sal","Cachoeira_do_Roncador"]
isOneOfThePlaces = lambda x: x in placesConsidered
vectorIsOneOfThePlaces = np.vectorize(isOneOfThePlaces)
filteredDataFrame = dataset[vectorIsOneOfThePlaces(dataset['place'])]

# Removes the Tufa sample
filteredDataFrame = filteredDataFrame.loc[filteredDataFrame['Porosity (%)']!=filteredDataFrame['Porosity (%)'].max()]
filteredDataFrame = filteredDataFrame.reset_index()

In [ ]:
# Calculates the mean spectra

workingDF = filteredDataFrame.groupby(["sample","place"]).mean()
workingDF.reset_index(level=["sample","place"], inplace=True)

In [ ]:
# Resample the dataset

dfRoncador = workingDF[workingDF['place'] == 'Cachoeira_do_Roncador']
dfPedreiraSal = workingDF[workingDF['place'] == 'Pedreira_Sal']  
nSamplesPedreiraSal = min(int(len(dfRoncador) * 2.5 //10),
                                len(dfPedreiraSal))
dfPedreiraSal = dfPedreiraSal.sample(n=nSamplesPedreiraSal, replace=False, random_state=1)

workingDF = pd.concat([dfPedreiraSal,dfRoncador]).reset_index()
workingDF = workingDF.drop(columns='index')    
    
XComplete = np.concatenate((workingDF.values[:,-1019:-1019].astype(np.float64),
                    workingDF.values[:,-969:-51].astype(np.float64)),
                    axis=1)
YComplete = workingDF['Porosity (%)'].values.astype(np.float64)

In [ ]:
# Dataset Normalization

mmX = MinMaxScaler()
mmY = MinMaxScaler()

XComplete = mmX.fit_transform(XComplete)
YComplete = mmY.fit_transform(YComplete.reshape(-1,1))

finalDataFrame = pd.DataFrame(XComplete)#,columns = myColumns)
finalDataFrame['place'] = workingDF['place']
finalDataFrame['poro'] = workingDF['Porosity (%)']

featureColumns = list(finalDataFrame.keys()[:-2])

In [ ]:
# Auxiliary Functions
upper_scores = None
def getKfoldIndexes():
  return copy.deepcopy(kfold_indexes)

def evaluateModel(model, modelName, saveBandImportances = False):
    global upper_scores
    scores = cross_validate(model, 
                            XComplete, 
                            y = np.ravel(YComplete),
                            cv=getKfoldIndexes(), 
                            scoring = {'mse': 'neg_mean_squared_error'},
                            return_estimator=True)
    upper_scores = scores
    #print('R2:',scores['test_r2'])
    print(modelName,'MSE',scores['test_mse'])
    generateGraphs(scores,modelName)
    if saveBandImportances:
        bandsImportance(scores['estimator'], modelName)
    
    return scores

def bandsImportance(estimatorsList, modelName):
    colunasComImportancia = filteredDataFrame.columns[58:-51].values
    importanceList = []
    
    cross_val_indexes = getKfoldIndexes()
    
    acumImportance = np.array([0]*918)
    foldCountIndex = 0
    for est, foldsSeparation in list(zip(estimatorsList,
                                         cross_val_indexes)):
        foldTrain, foldTest = foldsSeparation
        yFold = YComplete[foldTrain]
        xFold = XComplete[foldTrain]
        foldCountIndex = foldCountIndex + 1
        
        result = permutation_importance(est, xFold, yFold, n_repeats=10,
                                        random_state=0)
        print('Processing Fold:', foldCountIndex)
        acumImportance = acumImportance + result.importances_mean  
    
    acumImportance = list(map(lambda x: x/10, acumImportance))
    colunaZip = list(zip(colunasComImportancia, acumImportance))
    colunaZip = sorted(colunaZip, reverse=True, key=lambda x:x[1])
    with open('BandImportances' + modelName+'.txt', 'w') as file_object:
        file_object.write("Band;Importance\n")
        for bandImportance, importanceValue in colunaZip:
            file_object.write(str(bandImportance) + ";" + str(importanceValue) + "\n")

In [ ]:
# ML Preparation
listSamples = workingDF.groupby(["sample","place"]).mean()#.values[:,3:-1].astype(np.float64)
listSamples.reset_index(level=["sample","place"], inplace=True)    
listSamples = list(listSamples['sample'].values)

n_split = len(XComplete)

    

dictSampleIndexes = {}
for i in range(len(listSamples)):
    dictSampleIndexes[i] = list(workingDF[workingDF['sample'] == listSamples[i]].index)
    
kfold_indexes = list(KFold(n_split,shuffle=True).split(listSamples))

newKfoldIndexes = []

for train_fold, test_fold in kfold_indexes:
    listTrainFold = list(map(lambda x: dictSampleIndexes[x],train_fold))
    flatListTrain = [item for sublist in listTrainFold for item in sublist]
    
    listTestFold = list(map(lambda x: dictSampleIndexes[x],test_fold))
    flatListTest = [item for sublist in listTestFold for item in sublist]
    newKfoldIndexes.append((flatListTrain,flatListTest))
kfold_indexes = newKfoldIndexes


In [ ]:
# Error estimation plots

def generateGraphs(crosValidScores,modelName):
  resultList = crosValidScores['estimator']
  varia = 0
  cross_val_indexes = getKfoldIndexes()
  plt.style.use(['seaborn-ticks'])
  meanSquaredErrorsList = []
  listYhat = []
  listY = []
  for est in resultList:
    x_temp = cross_val_indexes[varia][1]
    if len(x_temp) > 0:
        ground_truth = YComplete[x_temp]
        x_temp = XComplete[x_temp]
        pred = est.predict(x_temp)
        listYhat = listYhat + list(pred)
        listY = listY + list(ground_truth.reshape(1,-1)[0])
        meanSquaredErrorsList.append(mean_squared_error(pred,
                                                        ground_truth.reshape(1,-1)[0]))
    else:
        print('Problem')
    varia = varia + 1

  plt.plot(listY, listYhat,"o")
  plt.plot([0, 1], [0, 1], 'k-')
  linear = LinearRegression()

  yArray = np.asarray(listY).reshape(len(listY),1)
  yHatArray = np.asarray(listYhat).reshape(len(listYhat),1)
  linear.fit(yArray,yHatArray)
  plt.plot(yArray, linear.predict(yArray),'k-', color='red')
 
    
  plt.xlabel('True Porosity')
  plt.ylabel(modelName+' Estimated Porosity')

  r2Result = r2_score(listY,listYhat)
  print("R2:", r2Result)
  print("MSE:", meanSquaredErrorsList)
  
  trueMSE = statistics.mean(meanSquaredErrorsList)
  print('mean:', trueMSE)
  print('std:',statistics.stdev(meanSquaredErrorsList))
  plt.text(0, 0.92, 'R2: '+ str(round(r2Result,4)) + '\nMSE: '+ str(round(trueMSE,6)), bbox=dict(facecolor='gray', alpha=0.5))
  plt.title(modelName)
  plt.grid(True)

  plt.show()
  

  errorsList = list(map(lambda x: x[0] - x[1], zip(listY,listYhat)))
  #extimateError(modelName, 
  #              'residual error', 
  #              errorsList,
  #              binsUse = [-.55,-.45,-.35,-.25,-.15,-.05,
  #                      .05,.15,.25,.35,.45,.55],
  #              normalFit=True)
  #
  #extimateError(modelName, 'MSE', meanSquaredErrorsList)

def extimateError(modelName, 
                  graphName, 
                  errorsList, 
                  binsUse = 10,
                  normalFit = False):
    
    
    # Fit a normal distribution to the data:
    
    plt.hist(errorsList, bins = binsUse, density=True)
    xmin, xmax = plt.xlim()
    ymin, ymax = plt.ylim()
    titleGraph = modelName + " " + graphName
    plt.title(titleGraph)
    if normalFit:
        
        shapTest = shapiro(errorsList)
        
        mu, std = norm.fit(errorsList)
        xGraph = np.linspace(xmin, xmax, 100)
        p = norm.pdf(xGraph, mu, std)
        plt.plot(xGraph, p, 'k', linewidth=2)
        textGraph = "Fit results: \nmu = %.2f \nstd = %.2f \nShapiro: %.4f \npval: %s" % (mu, std, shapTest[0], ('%.2E' if shapTest[1] < 0.0001 else '%.6f') % shapTest[1])
        plt.text(xmin,
                 ymax * 0.95, 
                 textGraph, 
                 bbox=dict(facecolor='gray', alpha=0.5),
                 horizontalalignment='left',
                 verticalalignment='top')
        print('Normal Fit\nShapiro:',
              shapTest[0],
              'pval:',
              shapTest[1])
    plt.grid()
    plt.show()


In [ ]:
# Linear Regression
linear = LinearRegression()
evaluateModel(linear,'Linear Reg')

In [ ]:
# SVR Model Evaluation
svr = SVR(gamma='scale', C=1.0, epsilon=0.05)
evaluateModel(svr, 'SVR', saveBandImportances=False)

In [ ]:
# KNN Model Evaluation
knn = KNeighborsRegressor(n_neighbors=3)
evaluateModel(knn,'KNN', saveBandImportances=False)

In [ ]:
# Random Forest
forest = RandomForestRegressor(n_estimators=10)
evaluateModel(forest, 'RF', saveBandImportances=False)

In [ ]:
# ANN Model Evaluation

def buildANN():
  c = Sequential()               
  c.add(Dense(units = 8,
              kernel_initializer = 'uniform',
              activation = 'sigmoid', 
              input_dim = len(XComplete[0,:])))
  c.add(Dense(units = 8, 
              kernel_initializer = 'uniform',
              activation = 'sigmoid'))
  c.add(Dense(units = 1, kernel_initializer = 'uniform',
              activation = 'sigmoid'))
  c.compile(optimizer = 'adam',
          loss = 'mean_squared_error', 
          metrics=['mse'])
  return c

ann = keras.wrappers.scikit_learn.KerasRegressor(buildANN, 
                                                 epochs=1600, 
                                                 batch_size = 1, 
                                                 verbose=2)
evaluateModel(ann, 'ANN', saveBandImportances=False)